In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import datasets
import IPython.display as display
from PIL import Image

from glob import glob

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
#구글 드라이브 연결
#이미지 파일 경로는 'gdrive/My Drive/base'
from google.colab import drive
drive.mount('/content/gdrive/')


In [ ]:
import os, shutil

base_dir = '/content/gdrive/My Drive/base'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')
train_acc_dir=os.path.join(train_dir, 'accident')
validation_acc_dir=os.path.join(validation_dir, 'accident')
test_acc_dir=os.path.join(test_dir, 'accident')
train_noacc_dir=os.path.join(train_dir, 'noaccident')
validation_noacc_dir=os.path.join(validation_dir, 'noaccident')
test_noacc_dir=os.path.join(test_dir, 'noaccident')


In [ ]:
#데이터 전처리
from keras.preprocessing.image import ImageDataGenerator

#모든 이미지를 1/255로 스케일 조정
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size= (160,160), batch_size = 20, class_mode = 'binary')
validation_generator = test_datagen.flow_from_directory(validation_dir, target_size= (160,160), batch_size = 20, class_mode = 'binary')


In [ ]:
#모델

IMG_SHAPE = (160,160,3)

base_model = tf.keras.applications.DenseNet201(
    include_top=False, weights='imagenet', input_tensor=None, input_shape=IMG_SHAPE,
    pooling=None, classes=1000
)

In [ ]:
base_model.trainable = False

In [ ]:
base_model.summary()

In [ ]:
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(1)
])

In [ ]:
################################
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
len(model.trainable_variables)

In [ ]:
history = model.fit_generator(train_generator, steps_per_epoch=40, epochs=30, validation_data=validation_generator,validation_steps=20)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()